In [13]:
import cmath
#from matplotlib.figure import Figure
import matplotlib.pyplot as plt    

from mymodules import intcode as itc


# Create class for Arcade unit. Contains all the methods that make the game run.
class ArcadeUnit:
    
    def __init__(self):
        
        # Instance of Intcode Computer
        self.computer = itc.IntcodeComputer()
        # Variables of the new frame to be drawn
        self.frame = [0, 0, 0, 0]
        # Dictionary of tile interpreters 
        self.objects = {0: 'empty', 1: 'wall tile', 2: 'block tile', 3: 'paddle', 4: 'ball'}
        # Global state of the game: a dictionary of all the tiles present in the game in the form
        # (X_COORD, Y_COORD): TILE_TYPE
        self.state = dict()
        # Score of the user
        self.score = 0
    
        return
    
    # Straight-forward name. Using try-except syntax since 'ball' and 'paddle' types
    # may not be present in the game until from the beginning 
    def get_ball_position(self):
        
        try:
            x = [key[0] for key in self.state if self.state[key] == 'ball'][0]
        except:
            x = 0
        
        return x
    
    
    def get_paddle_position(self):
        
        try:
            x = [key[0] for key in self.state if self.state[key] == 'paddle'][0]
        except:
            x = 0        
        
        return x
    
    
    def print_score(self):
        
        print("User score: ", self.score)
        
        return 
    
    
    # Compute the next frame. The instruction to move the paddle is calculated as a difference 
    # between the paddle and the ball position
    def compute_frame(self):
         
        x_pad = self.get_paddle_position()
        x_bal = self.get_ball_position()
        
        # 0-> stay put, -1 -> move left, 1-> move right
        move = 0 if x_pad == x_bal else (x_bal-x_pad)//abs(x_pad - x_bal)
        
        # Run computer for 3 times and get output. We can pass input every time
        # since if no opcode=3 is encountered, input is ignored
        for i in range(3): 
            
            output = self.computer.run(move)
            
            if output == 'end':
                return False
            
            else:
                self.frame[i] = output
                
        self.frame[3] += 1
        
        return True
    
    # Draw frame or user score. Draw in this case means update the 'state' variable of the
    # arcade unit. For actual drawing of the game to screen, see 'print_screen()' function later.
    def draw_frame(self):
        
        if self.frame[0] == -1:
            self.score = self.frame[2]
            #print("*** SCORE ***: ", self.score)
        
        else:
            self.state[self.frame[0], self.frame[1]] = self.objects[self.frame[2]]
     
        return
    
    
    # Get back the total number of block tiles left in the game
    def blocks_count(self):
        
        blocks = 0
        
        for key in self.state:
            if self.state[key] == 'block tile':
                blocks += 1
        
        return blocks
    
    
    # Run the game until opcode=99 -> game over
    def run_game(self, quarters = None):
        
        game_over = False
        
        while not game_over:
            
            if self.compute_frame():
                self.draw_frame()
                
                if self.frame[3]%100 == 0:
                    self.print_screen()
            
            else:
                game_over = True
                
        return 0
       
    
    # Print to screen informations about the current position of theball and the paddle.
    # USeful for debugging
    def get_game_info(self):
        
        ball_pos = [key for key in self.state if self.state[key] == 'ball']
        print("Ball is in position ", ball_pos)
        paddle_pos = [key for key in self.state if self.state[key] == 'paddle']
        print("Paddle is in position ", paddle_pos)
        
        return
    
    
    # Print the game state on the screen.It is done as a scatter plot with different
    # markers for different tiles. Paddle is a red triangle.
    def print_screen(self):
        
        plt.figure(figsize=(8, 8))
        
        syms = {'wall tile': 's', 'paddle': 6, 'ball': '8', 'block tile':'X'}
        cols = {'wall tile': 'black', 'paddle': 'red', 'ball': 'blue', 'block tile':'green'}
        sizes = {'wall tile': 50, 'paddle': 200, 'ball': 200, 'block tile': 40}
       
        
        for key in syms:
            x = []
            y = []
            col = cols[key]
            mark = syms[key]
            size = sizes[key]
            
            for o_key in self.state:
                
                if self.state[o_key] == key:
                    
                    x.append(o_key[0])
                    y.append(- o_key[1])
    
            plt.scatter(x, y, c = col, s = size, marker = mark)
    
        #plt.savefig("imgs/game_frame"+str(self.frame[3]))
        #plt.show()
        plt.close('all')
            
        return

In [14]:
intcode = []

with open('input.txt', 'r') as infile:
    intcode = infile.readline().replace('\n', '').split(',')
    
intcode = [int(x) for x in intcode]
intcode[0] = 2

In [15]:
au = ArcadeUnit()
au.computer.load_intcode(intcode)

au.run_game()
print("There are ", au.blocks_count(), " left blocks")
au.print_score()
au.print_screen()


Program is finished. Haulting.
There are  0  left blocks
User score:  10776
